In [15]:
# Mã hoá nguồn bằng phương pháp nén văn bản thành mã nhị phân sử dụng Run length encoding để giảm bit
def run_length_encoding(text):
    encoded_text = ""
    i = 0
    while i < len(text):
        count = 1
        while i + 1 < len(text) and text[i] == text[i + 1]:
            count += 1
            i += 1
        encoded_text += str(count) + text[i]
        i += 1
    return encoded_text

with open('text.txt', 'r') as file:
    text = file.read()
    encoded_text = run_length_encoding(text)
    binary_string = ''.join(format(ord(i), '08b') for i in encoded_text)
    print(binary_string)


0011000101101000001100010110010100110010011011000011000101101111


In [16]:
# Mã hoá kênh dùng mã phát hiện lỗi CRC
import binascii

def crc_encode(binary_string):
    data = int(binary_string, 2).to_bytes((len(binary_string) + 7) // 8, byteorder='big')
    crc = binascii.crc32(data)
    crc_binary = format(crc, '032b')
    return binary_string + crc_binary

CRC_binary_string = crc_encode(binary_string)

# In ra
print(CRC_binary_string)

001100010110100000110001011001010011001001101100001100010110111100001010010101111101101100101000


In [17]:
# Định nghĩa chuỗi tiền tố và hậu tố
preamble = '010101'
postamble = '101010'

# Thêm tiền tố và hậu tố vào dòng bit
bits_with_pre_and_post = preamble + CRC_binary_string + postamble

#Tín hiệu sau khi thêm tiền tố và hậu tố
print(bits_with_pre_and_post)

010101001100010110100000110001011001010011001001101100001100010110111100001010010101111101101100101000101010


In [18]:
# Điều chế BFSK 
import numpy as np
from scipy.io.wavfile import write

def encode(binary_string, sample_rate=44100, baud_rate=50):
    samples_per_bit = sample_rate // baud_rate
    audio_signal = np.zeros(samples_per_bit * len(binary_string))
    time = np.linspace(0, 1, samples_per_bit)

# Sóng mang có tần số F0 = 5000 Hz và F1 = 15000 Hz
    for i in range(len(binary_string)):
        if binary_string[i] == '1':
            audio_signal[i*samples_per_bit:(i+1)*samples_per_bit] = np.sin(2 * np.pi * 300 * time)
        elif binary_string[i] == '0':
            audio_signal[i*samples_per_bit:(i+1)*samples_per_bit] = np.sin(2 * np.pi * 100 * time)
    return audio_signal

audio_signal = encode(bits_with_pre_and_post)
write('audio_signal.wav', 44100, audio_signal)

In [19]:
import pyaudio
from scipy.io.wavfile import read

# Đọc tín hiệu âm thanh từ một tệp
sample_rate, audio_signal = read('audio_signal.wav')

# Chuyển đổi tín hiệu âm thanh thành một mảng numpy
audio_signal = np.array(audio_signal, dtype=np.float32)

# Khởi tạo PyAudio
p = pyaudio.PyAudio()

# Mở một luồng để phát âm thanh
stream = p.open(format=pyaudio.paFloat32,
                channels=1,
                rate=sample_rate,
                output=True)

# Phát tín hiệu âm thanh
stream.write(audio_signal.tobytes())

# Đóng luồng và kết thúc PyAudio
stream.stop_stream()
stream.close()
p.terminate()